# StarPlus fMRI Data Exploration


## Functional Magnetic Resonance Imaging

* Functional Magnetic Resonance Imaging (fMRI) is a technique for obtaining three-dimensional images related to neural 
activity in the brain through time. 
* It measures the ratio of oxygenated haemoglobin to deoxygenated haemoglobin in the blood with respect to a control
baseline, at many individual locations with the brain.
* It is widely believed that blood oxygen level is influenced by local neural activity, and hence this blood oxygen
level dependent (BOLD) response is generally taken as an indicator of neural activity. 
* An fMRI scanner measures the value of the fMRI signal (BOLD response) at all points in a three dimensional grid, or
image.
* We refer to cells within this 3D image as *voxels* (volumetric elements of pixels).
* The voxels in a typical fMRI study has a volume of a few tens of cubic millimeters, and a typical 3D brain image
typically contains 10,000 to 15,000 voxels which contain cortical matter and are thus of interest. 

## StarPlus Dataset Study Notes

* Subjects experienced a collection of trials.
* During each trial, thhey were shown in sequence a sentence and a simple picture, then answered whether the sentence 
correctly described the picture.
* Half the trials, the sentence was presented first and the other half the picture was presented first. 
* The stimulus timeline:
    * First $\Rightarrow$ 4 seconds `(image = 1)`
    * Blank screen $\Rightarrow$ 4 seconds `(image = 9)`
    * Second $\Rightarrow$ up to 4 seconds (ending when subjected pressed the mouse button) `(image = 17)`
    * Rest $\Rightarrow$ 15 seconds(about 30 images)
* Each trial lasted about $\approx$ 27 seconds.
* Each subject undertook 40 trials.
    * There were interspersals of 10 additional rest periods.
* fMRI images were collected every 500 msec.
* The data is partitioned into trials. 
    * For some of these intervals, the subject simply rested, or gazed at a fixation point on the screen.
* The data is marked up with 25-30 anatomically defined regions (called "Regions of Interest", or ROIs).

[1\]

In [1]:
# ========== # IMPORTS # ========== #

import scipy.io as sio
import numpy as np


### Notes about StarPlus dataset:
- Each data file is for a subject with the subject id in as the integer in the title
- The files are marked to be loaded in Matlab 7

In [2]:
# ========== # LOADING DATA # ========== #
test = sio.loadmat('../data/data-starplus-04847-v7.mat')

info = test['info'][0]  # (1,54)
data = test['data']   # (54,1)
meta = test['meta']      # (1,1)

print('Mat Structure')
sio.whosmat('../data/data-starplus-04847-v7.mat')

Mat Structure


[('info', (1, 54), 'struct'),
 ('data', (54, 1), 'cell'),
 ('meta', (1, 1), 'struct')]

## MATLAB Dataset Notes  

##### Output
```
meta = 
  struct with fields:
         colToCoord: [4698×3 double]
         coordToCol: [64×64×8 double]
            nvoxels: 4698
               dimx: 64
               dimy: 64
               dimz: 8
            ntrials: 54
              study: 'data-starplus'
    testReadingSpan: 2
     testVandenburg: [13.5000 15]
      testPaperFold: [14 1]
    testSurfaceFold: [47 3]
            subject: '04847'
                roi: 'CALC_LFEF_LIPL_LIT_LPPREC_LSPL_LTRIA_RFEF_RIPS_ROPER_RSGA_RT_SMA_LDLPFC_LIPS_LOPER_LSGA_LT_RDLPFC_RIPL_RIT_RPPREC_RSPL_RTRIA'
         nsnapshots: 2800
               rois: [1×25 struct]
           colToROI: {4698×1 cell}


info = 
  1×54 struct array with fields:
    cond
    len
    mint
    maxt
    missing
    nmissing
    sentence
    img
    sentenceRel
    sentenceSym1
    sentenceSym2
    actionRT
    actionAnswer
    firstStimulus


data =
  54×1 cell array
    {13×4698 double}
    {55×4698 double}
    {55×4698 double}
    ...
    {54×4698 double}
    {55×4698 double}
    {17×4698 double}
```


* Each voxel dimension is `(64,64,8)`
* Each subject has `ntrials = 54`
* The number of images in the dataset `nvoxels`
* The number of voxels in each image is `nsnapshots`
* `colToCoord(v,:)` gives the geometric coordinate `(x,y,z)` of the voxel corresponding to column `v` in the data
    * `(4634,3)` matrix of `double`
* `coordToCol(x,y,z)` gives the column index (within the data) of the voxel whose coordinate is `(x,y,z)`
    * `(64,64,8)` matrix of `double`
* 


In [36]:
# ========== # SANITY CHECKS # ========== #

print('|=====| info |=====|')
print(test['info'].__class__)
# print(test['info'].dtype)
print(test['info'].dtype.names)

print('|=====| data |=====|')
print(test['data'].__class__)
# print(test['data'].dtype)
print(test['data'].dtype.names)

print('|=====| meta |=====|')
print(test['meta'].__class__)
# print(test['meta'].dtype)
print(test['meta'].dtype.names)

print(info.shape)
print(data.shape)
print(meta.shape)


|=====| info |=====|
<class 'numpy.ndarray'>
('cond', 'len', 'mint', 'maxt', 'missing', 'nmissing', 'sentence', 'img', 'sentenceRel', 'sentenceSym1', 'sentenceSym2', 'actionRT', 'actionAnswer', 'firstStimulus')
|=====| data |=====|
<class 'numpy.ndarray'>
None
|=====| meta |=====|
<class 'numpy.ndarray'>
('colToCoord', 'coordToCol', 'nvoxels', 'dimx', 'dimy', 'dimz', 'ntrials', 'study', 'testReadingSpan', 'testVandenburg', 'testPaperFold', 'testSurfaceFold', 'subject', 'roi', 'nsnapshots', 'rois', 'colToROI')
(54,)
(1,)
(1, 1)


In [3]:
print(info.dtype.names)

print()

# for i in info[18]:
print(info[18]['mint'])

('cond', 'len', 'mint', 'maxt', 'missing', 'nmissing', 'sentence', 'img', 'sentenceRel', 'sentenceSym1', 'sentenceSym2', 'actionRT', 'actionAnswer', 'firstStimulus')

[[949]]


In [4]:
print(data.shape)
# data.shape -> (54,1)
t1 = data[17,0]   # x = 18th trial

print(t1.shape)
# data{x}(t,v) -> fMRI observation at voxel v, at time t within trial x
print(t1[0,0:9])  # data{18}(1,1:10) -> data[x,0][t,v]
print(t1[0,:])  # returns the full image at time 0

(54, 1)
(55, 4698)
[ 4.04726781  2.75427747  0.56162819 -0.09324056 -0.42475937  5.00742041
  0.91386166 -2.08359839 -3.96658501]
[ 4.04726781  2.75427747  0.56162819 ... -7.21588547  2.5284988
  0.89947608]


## References

[1\] T. M. Mitchell et al., “Learning to Decode Cognitive States from Brain Images,” Machine Learning, vol. 57, no. 1, pp. 145–175, Oct. 2004.
